In [1]:
import pandas as pd

from nltk.corpus import stopwords as sw
from multiprocessing import Pool
from gensim.models.phrases import Phrases
from gensim.corpora import Dictionary
import pymorphy2
import re

In [ ]:
morph = pymorphy2.MorphAnalyzer()
token_pattern = re.compile(r'\b[\w\.\/]+\b')
tokenize = lambda doc: token_pattern.findall(doc)

badlist = set(['отличный', 'идеальный', 'состояние', 'продажа', 'продавать', 'обменять', 
               'москва', 'продать', 'хороший', 'продаваться', 'супер'])

def get_tokens(line):
    cur_line = tokenize(line)
    normalized_tokens = [morph.parse(x)[0].normal_form for x in cur_line]
    filtered_tokens = [token for token in normalized_tokens if not token in badlist and not token in stopwords]  
    return filtered_tokens

In [ ]:
pool = Pool()
df_searchquery['norm_query'] = pool.map(get_tokens, df_searchquery['query'].values)
pool.terminate()

In [ ]:
bigram_transformer = Phrases(df_searchquery['norm_query'].values)
df_searchquery['norm_query'] = [bigram_transformer[q] for q in df_searchquery['norm_query']]